In [ ]:
using Pkg; Pkg.activate("."); Pkg.instantiate()
# ----------------------------------------------------------------/
using OrdinaryDiffEq, ModelingToolkit, DataDrivenDiffEq
using LinearAlgebra, ComponentArrays, Random
# ----------------------------------------------------------------/
#OptimizationFlux for ADAM and OptimizationOptimJL for BFGS
using Optimization, OptimizationOptimisers, OptimizationOptimJL
using DiffEqSensitivity, Lux
# ----------------------------------------------------------------/
#using JLD2, FileIO
using Statistics, JLD2, LaTeXStrings, Plots, Interact
gr()
theme(:bright)

In [ ]:
# Set a random seed for reproduceable behaviour
rng = Random.default_rng()
Random.seed!(1234)

# Charged classical particle in a homogeneous magnetic field

## Model

In [ ]:
function cyclotron!(du, u, p, t)
    α, β, ϕx, ϕy = p
    du[1] =  β*u[2] + u[3]
    du[2] = -β*u[1] + u[4] 
    du[3] =  β*u[4] - (β^2 + α)*u[1] + ϕx
    du[4] = -β*u[3] - (β^2 + α)*u[2] + ϕy
end

# Solve perturbed motion

In [ ]:
# Interval of integration and size of steps
tspan = (0, 19.9)
dx = 1e-2
# Initial contidions: x, y, px, py,
u0 = [0, 0, 1.5, 0]
#= Parameters:
    amplitude of the potential field
    amplitude of the magnetic field
    external force aligned in Ox and Oy
=#
# Perturbed parameters
p̃ = [2, 3, 0, 1]
# -------------------------------------------------/
# Perturbed solution
perturbed_solution = solve(
    ODEProblem(cyclotron!, u0, tspan, p̃),
    RK4(),
    abstol=1e-6,
    reltol=1e-6,
    saveat=dx)
# Create an array for the perturbed solution
X̃ = Array(perturbed_solution)
t̃ = perturbed_solution.t
# -------------------------------------------------/
# Plots
# Perturbed solution
plot(X̃[1,:], X̃[2,:], style=:solid, color="#13B9D3", lw=6, label=nothing)
# Initial condition
scatter!([X̃[1,1]], [X̃[2,1]], color="#3CA2D8", ms=9, label=nothing)
# Plot attributes and settings
plot!(xlabel=L"x(t)", ylabel=L"y(t)",
    size=(800, 800),
    margin=2mm,
    xtickfont=font(12),
    ytickfont=font(12),
    guidefont=font(18),
    framestyle=:box)
savefig("cyclotron/fullCyclotron.pdf")

### Incomplete cycles for data generation

In [ ]:
# Interval of integration and size of steps
tspan = (0, 5)
# -------------------------------------------------/
# Perturbed solution
dx=5e-2
perturbed_solution = solve(
    ODEProblem(cyclotron!, u0, tspan, p̃),
    RK4(),
    abstol=1e-6,
    reltol=1e-6,
    saveat=dx)
# Create an array for the perturbed solution
X̃ = Array(perturbed_solution)
t̃ = perturbed_solution.t
# -------------------------------------------------/
# Plots
# Perturbed solution
plot(X̃[1,:], X̃[2,:], style=:solid, color="#13B9D3", lw=5, label=nothing)
# Initial condition
scatter!([X̃[1,1]], [X̃[2,1]], color="#3CA2D8", ms=8, label=nothing)
# Plot attributes and settings
plot!(xlabel=L"x", ylabel=L"y",
    size=(600,600),
    margin=2mm,
    xtickfont=font(11),
    ytickfont=font(11),
    guidefont=font(16),
    framestyle=:box)
#savefig("cyclotron/classicalCyclotron.pdf")

### Generate noisy data

In [ ]:
σ = 1e-3
μ = mean(X̃, dims=4)
X̂ = X̃ .+ (σ*μ) .* randn(eltype(X̃), size(X̃))

@save "noisydata_cyclotron.jld2" X̂

# Plots
plotData = plot(X̃[1,:], X̃[2,:], color="#13B9D3", lw=6, label=nothing)
scatter!([X̃[1,1]], [X̃[2,1]], color="#680AC7", ms=8, label=nothing)
scatter!(transpose(X̂[1,:]), transpose(X̂[2,:]),
    color="#724A83",
    alpha=0.75,
    ms=7,
    label=nothing)
plot!(xlabel=L"x(t)", ylabel=L"y(t)",
    size=(600,600),
    margin=2mm,
    xtickfont=font(12),
    ytickfont=font(12),
    guidefont=font(16),
    framestyle=:box)
savefig("cyclotron/classicalCyclotron_noisy.pdf")

### Animation (optional)

In [ ]:
#=
anim = @animate for i in 1:length(X̃[1,:])
    plot(X̃[1,:], X̃[2,:], color="#00C1EC", alpha=0.5, lw=5, label=nothing)
    plot!(xlabel=L"x", ylabel=L"y",
        size=(600,600),
        margin=5mm,
        xtickfont=font(11), 
        ytickfont=font(11),
        guidefont=font(15),
        framestyle=:semi
    )
    scatter!([X̃[1,i]], [X̃[2,i]],
        color="#F97489",
        ms=12,
        label=nothing
    )
end

gif(anim, fps=24)
=#

# Define neural network for UDE parameterisation

In [ ]:
n = 10

# Define network
U = Lux.Chain(
        Lux.Dense(4, n, tanh),
        Lux.Dense(n, n, tanh),
        Lux.Dense(n, n, tanh),
        Lux.Dense(n, 4))

# Get the initial parameters and state variables of the model
p_nn, state_vars = Lux.setup(rng, U) .|> gpu

function ude_dynamics!(du, u, p, t, p_true)
    # Network function
    û = U(u, p, state_vars)[1]
    # Parameters
    α, β, ϕx, ϕy = p_true
    # ODE system
    du[1] =  β*u[2] + u[3] 
    du[2] = -β*u[1] + u[4]
    du[3] =  β*u[4] + û[1] 
    du[4] = -β*u[3] - û[2]
end

nn_dynamics!(du, u, p_nn, t) = ude_dynamics!(du, u, p_nn, t̃, p̃)

# Define the problem for NNs --- X̂[:,1] is the initial datum
problem_nn = ODEProblem(nn_dynamics!, X̂[:,1], tspan, p_nn)

# Define a predictor
function predict(θ, X=X̂[:,1], T=t̃)
    problem_predict = remake(problem_nn, u0=X, tspan=(T[1], T[end]), p=θ)
    Array(
        solve(
            problem_predict,
            Tsit5(), 
            saveat=T,
            abstol=1e-6,
            reltol=1e-6,
            sensealg=ForwardDiffSensitivity()
        )
    )
end

# Simple L₂ loss
function loss(θ)
    sum(abs2, X̂ .- predict(θ))
end

# Callback options
losses = Float64[]
callback = function (p, l)
    push!(losses, l)
    if length(losses) % 50 == 0
        println("Epochs: $(length(losses)) -- Current loss: $(losses[end])")
    end
    return false
end

# Training

In [ ]:
M, N = 500, 1000

# Optimisation function
opfun = Optimization.OptimizationFunction((x, p) -> loss(x), Optimization.AutoZygote())

# Optimisation block with ADAM
println("Training with ADAM")
ADAM_ = Optimization.OptimizationProblem(opfun, ComponentVector{Float64}(p_nn))
rADAM = Optimization.solve(ADAM_, Adam(0.1), callback=callback, maxiters=M)

# Optimisation block with BFGS
println("Training with BFGS")
BFGS_ = Optimization.OptimizationProblem(opfun, rADAM.minimizer)
sBFGS = Optimization.solve(BFGS1, Optim.BFGS(initial_stepnorm=0.01), callback=callback, maxiters=N)

In [ ]:
# Plots
plot(1:M, losses[1:M],
    yaxis=:log10, xaxis=:log10,
    xlabel="Iterations",
    ylabel="Loss",
    label="ADAM",
    color="#0E9BEB",
    lw=4)
plot!(M:length(losses), losses[M:end],
    yaxis=:log10, xaxis=:log10,
    label="BFGS",
    color="#A284FE",
    lw=4)
plot!(
    margin=5mm,
    size=(800,500),
    xtickfont=font(12),
    ytickfont=font(12),
    guidefont=font(16),
    framestyle=:box)
savefig("cyclotron/minimisationRate.pdf")

## Prediction and analysis of solutions

In [ ]:
p̄ = sBFGS.minimizer
X̄ = predict(p̄, X̂[:,1], t̃)
Ȳ = U(X̄, p̄, state_vars)[1]

@save "cyclotron/trainedParameters.jld2" p̄ X̄ Ȳ

# Noisy data vs learned solution
# Plots
plotLearn = plot(X̄[1,:], X̄[2,:], color="#132DBD", lw=6, label=nothing)
scatter!(transpose(X̂[1,:]), transpose(X̂[2,:]),
    color="#6FCE75",
    msc="#6FCE75",
    alpha=0.5,
    ms=10,
    label=nothing)
plot!(xlabel=L"x(t)", ylabel=L"y(t)",
    size=(700,700),
    margin=4mm,
    xtickfont=font(12),
    ytickfont=font(12),
    guidefont=font(16),
    framestyle=:box)
savefig("cyclotron/learnedCyclotron.pdf")

# Dynamical reconstruction

In [ ]:
function dynReconstruction!(du, u, p0, t)
    # Prediction from the network
    û = nn_ude(u, p0)
    # True parameters
    α, β, ϕx, ϕy = p̃
    # ODE system
    du[1] =  β*u[2] + u[3] 
    du[2] = -β*u[1] + u[4]
    du[3] =  β*u[4] + û[1] 
    du[4] = -β*u[3] - û[2]
end

# Create a Basis
@variables u[1:4]

# Generate the basis functions, multivariate polynomials up to deg 5 and sine
b = [polynomial_basis(u, 5); cos.(u)]
basis = Basis(b, u);

# Create the thresholds for seeking purposes
λ = exp10.(-2:1e-2:2.5)

# Create an optimizer for the SINDy problem
opt = STLSQ(λ)

# Test on ideal derivative data for unknown function
println("Sparse regression")
nn_ude = solve(
            DirectDataDrivenProblem(X̄, Ȳ),
            basis, 
            opt, 
            progress=true, 
            sampler=DataSampler(Batcher(n=5, shuffle=true, repeated=true))
)

println(parameters(nn_ude))

# Solve the dynamical hybrid model 
nn_solution = solve(
                ODEProblem(dynReconstruction!, u0, tspan, parameters(nn_ude)), 
                Tsit5(),
                saveat=t̃)

In [ ]:
# Plot
Ỹ = Array(nn_solution)
plot(X̃[4,:], lw=4, color="#FB65AA", label="true")
plot!(Ỹ[4,:], lw=0, markershape=:circle, ms=6, color="#33C3D3", label="learned")
plot!(xtickfont=font(12), ytickfont=font(12), guidefont=font(16), size=(600, 400), framestyle=:box)
#savefig("cyclotron/reconstructedDynamics.pdf")

In [ ]:
@save "cyclotron/reconstructedParameters.jld2" nn_ude

# Long-term prediction

In [ ]:
dτ = 1e-2
t_long = (0, 19.9)

lt_ode_pre = ODEProblem(dynReconstruction!, X̄[:,1], t_long, parameters(nn_ude))
lt_predict = solve(lt_ode_pre, Rosenbrock23(), abstol=1e-6, reltol=1e-6, saveat=dτ)

lt_ode_true = ODEProblem(cyclotron!, u0, t_long, p̃)
lt_solution = solve(lt_ode_true, Rosenbrock23(), saveat=lt_predict.t)

In [ ]:
S = Array(lt_solution)
Σ = Array(lt_predict)

plotLT = plot(S[1,:], S[2,:], color="#69409e", alpha=1, lw=6, label="long-term solution")
plot!(Σ[1,:], Σ[2,:], color="#36c2f3", alpha=1, lw=8, style=:dashdot, label="long-term prediction")
scatter!(X̃[1,:], X̃[2,:], color="#87CE51", alpha=1.0, ms=8, label="Training data")

plot!(xlabel=L"x(t)", ylabel=L"y(t)",
    margin=3mm,
    xtickfont=font(12), 
    ytickfont=font(12), 
    guidefont=font(16),
    legendfont=font(10),
    size=(800,800),
    framestyle=:box)
savefig("cyclotron/longtermCyclotron.pdf")